In [222]:
import pandas as pd
import numpy as np
from collections import namedtuple, defaultdict
from queue import deque
from heapq import heappop, heappush

In [205]:
wiki_pages = pd.read_csv('simple_english_wiki_pages.csv')
pagelinks = pd.read_csv('simple_english_wiki_pagelinks.csv')

In [207]:
full_df = wiki_pages.merge(pagelinks, how='right', left_on='page_id', right_on='pl_from')

In [209]:
full_df.page_title = full_df.page_title.astype(str)
full_df.pl_title = full_df.pl_title.astype(str)

In [210]:
Edge = namedtuple('Edge', 'src dst')

class DirectedGraph:
    
    def __init__(self):
        self.neighbors = defaultdict(set)
        self.nodes = set()
        
    def add_edge(self, edge: Edge):
        self.neighbors[edge.src].add(edge.dst)
        self.nodes.add(edge.src)
        self.nodes.add(edge.dst)
            
    def __getitem__(self, item):
        return self.neighbors.get(item)

In [211]:
g = DirectedGraph()
for i in range(len(full_df)):
    g.add_edge(Edge(full_df.page_title[i], full_df.pl_title[i]))

In [212]:
g['Analytics']

{'Computer_programming', 'Data', 'Statistics'}

In [215]:
def bfs(graph: DirectedGraph, start: str):
    
    seen = set()
    v_queue = deque()
    v_queue.append((start, 0))
    result = []
    while v_queue:
        current, current_level = v_queue.popleft()
        if graph[current] is not None:
            children = [edge for edge in graph[current]]
        else:
            continue
        for child in children:
            if child not in seen:
                seen.add(child)
                child_level = (child, current_level + 1)
                v_queue.append(child_level)
                result.append(child_level)
                
    return result

In [233]:
# Задача №1

answer = bfs(g, 'Analytics')
for i in answer:
    if i[0] == 'Algorithm':
        print(i)

('Algorithm', 4)


In [217]:
# Задача №2

def bfs_second_task(graph: DirectedGraph, start: str):
    seen = set()
    v_queue = deque()
    v_queue.append((start, 0))
    result = 0
    while v_queue:
        current, current_level = v_queue.popleft()
        if graph[current] is not None and 'Algorithm' in graph[current]:
            result += 1
            return result
        else:
            return result

In [234]:
potential_articles = []
for i in answer:
    if i[1] == 3:
        potential_articles.append(i[0])

status = []
for i in potential_articles:
    status.append(bfs_second_task(g, i))

solution_task2 = pd.DataFrame(zip(potential_articles, status), columns = ['potential_articles', 'status'])
answer_task2 = solution_task2[solution_task2.status == 1].potential_articles.values.tolist()[0]
answer_task2

'Computer_science'

In [235]:
# Задача №3

WeightedEdge = namedtuple('Edge', 'src dst distance')

class WeightedDirectedGraph:
    
    def __init__(self):
        self.neighbors = defaultdict(set)
        self.nodes = set()
        
    def add_edge(self, edge: WeightedEdge):
        self.neighbors[edge.src].add(edge)
        self.nodes.add(edge.src)
        self.nodes.add(edge.dst)
            
    def __getitem__(self, item):
        return self.neighbors.get(item, [])



In [236]:
wg = WeightedDirectedGraph()

for i in range(len(full_df)):
    wg.add_edge(WeightedEdge(full_df.page_title[i], full_df.pl_title[i], len(full_df.pl_title[i])))

In [227]:
def dijkstra(graph: WeightedDirectedGraph, start):
    heap, seen_vertices, min_distances = [(0, start)], set(), {start: 0}
    steps = []
    while heap: # O(V)
        curr_distance, current_vertex = heappop(heap) #O(logV) # start in vertex with min score
        if current_vertex not in seen_vertices:
            
            for _, next_vertex, distance in graph.neighbors.get(current_vertex, []): # O(E)
                if next_vertex in seen_vertices:
                    continue
                    
                prev_min_distance = min_distances.get(next_vertex)
                new_distance = curr_distance + distance
                
                if prev_min_distance is None or new_distance < prev_min_distance:
                    min_distances[next_vertex] = new_distance
                    heappush(heap, (new_distance, next_vertex)) # O(logV)
                    
            seen_vertices.add(current_vertex)

    return min_distances

In [229]:
dists = dijkstra(wg, 'Analytics')

In [250]:
answer_task3 = dists['Algorithm']
answer_task3

29

In [247]:
# Задача №4

def dijkstra_fourth_task(graph: WeightedDirectedGraph, start):
    heap, seen_vertices, min_distances = [(0, start)], set(), {start: (0, start)}
    steps = []
    while heap: # O(V)
        curr_distance, current_vertex = heappop(heap) #O(logV) # start in vertex with min score
        if current_vertex not in seen_vertices:
            
            for _, next_vertex, distance in graph.neighbors.get(current_vertex, []): # O(E)
                if next_vertex in seen_vertices:
                    continue
                
                if min_distances.get(next_vertex) is not None:
                    prev_min_distance = min_distances.get(next_vertex)[0]
                else:
                    prev_min_distance = None
                
                new_distance = curr_distance + distance
                
                if prev_min_distance is None or new_distance < prev_min_distance:
                    min_distances[next_vertex] = (new_distance, current_vertex)
                    heappush(heap, (new_distance, next_vertex)) # O(logV)
                    
            seen_vertices.add(current_vertex)

    return min_distances

In [251]:
dists_fourth_task = dijkstra_fourth_task(wg, 'Analytics')
answer_task4 = dists_fourth_task['Algorithm'][1]
answer_task4

'Logic'